In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def get_data(token):
    res = requests.get(
        f'https://api.cryptowat.ch/markets/coinbase-pro/{token}usd/ohlc',
        params={
            'periods': '3600',
            'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
        }
    )
    
    if res.json().get('error',0)!=0:
        res = requests.get(
            f'https://api.cryptowat.ch/markets/kraken/{token}usd/ohlc',
            params={
                'periods': '3600',
                'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
            }
        )
        

    df = pd.DataFrame(
        res.json()['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
    df['ts'] = pd.to_datetime(df.ts, unit='s')
    df['token'] = token
    
    return df

In [3]:
tokens = ['ETH', 'SOL', 'AVAX', 'USDT', 'FLOW']


In [4]:
dfs = [get_data(token) for token in tokens]
df = pd.concat(dfs,axis=0)

In [7]:
Volume_USD = df.groupby('token')['volumeUSD'].agg(np.sum).sort_values(ascending = False).to_frame()

In [10]:
Volume_USD

,volumeUSD
token,
ETH,2.236559e+10
SOL,7.326824e+09
AVAX,3.691328e+09
USDT,2.056140e+09
FLOW,9.005689e+07


In [89]:
df=pd.concat(dfs).set_index('ts')

In [116]:
close_ratio_ETH_BY_SOL = df.groupby('ts', as_index=False).apply(lambda x: x[x['token'] == 'ETH'].close/x[x['token'] == 'SOL'].close)
    

In [117]:
close_ratio_ETH_BY_SOL.head().to_frame()

,,close
,ts,
0,2021-11-22 00:00:00,18.471694
1,2021-11-22 01:00:00,18.736150
2,2021-11-22 02:00:00,18.577985
3,2021-11-22 03:00:00,18.794937
4,2021-11-22 04:00:00,18.891059


In [118]:
ratio = close_ratio_ETH_BY_SOL

In [119]:
ratio.index=ratio.index.droplevel(0)

In [121]:
ratio = ratio.head().to_frame()

In [122]:
ratio.rename(columns={"close":"ratio"},inplace=True) 

In [123]:
df = df.join(ratio, on='ts')
df.head()

,open,high,low,close,volume,volumeUSD,token,ratio
ts,,,,,,,,
2021-11-22 00:00:00,4317.98,4342.24,4246.07,4262.99,7262.562789,3.123077e+07,ETH,18.471694
2021-11-22 01:00:00,4263.04,4270.34,4212.45,4234.37,8437.946084,3.574586e+07,ETH,18.736150
2021-11-22 02:00:00,4234.87,4246.72,4171.17,4217.89,9259.725370,3.895639e+07,ETH,18.577985
2021-11-22 03:00:00,4217.88,4223.48,4163.58,4193.47,9259.899519,3.882032e+07,ETH,18.794937
2021-11-22 04:00:00,4192.95,4213.59,4147.00,4168.35,7934.546906,3.315476e+07,ETH,18.891059


In [73]:
df.rename(columns = {
    'volume':'volumeBase',
    'volumeUSD':'volumeTerm'},
          
     inplace = True)

In [74]:
df

,open,high,low,close,volumeBase,volumeTerm,token
ts,,,,,,,
2021-11-22 00:00:00,4317.980,4342.240,4246.070,4262.990,7262.562789,3.123077e+07,ETH
2021-11-22 01:00:00,4263.040,4270.340,4212.450,4234.370,8437.946084,3.574586e+07,ETH
2021-11-22 02:00:00,4234.870,4246.720,4171.170,4217.890,9259.725370,3.895639e+07,ETH
2021-11-22 03:00:00,4217.880,4223.480,4163.580,4193.470,9259.899519,3.882032e+07,ETH
2021-11-22 04:00:00,4192.950,4213.590,4147.000,4168.350,7934.546906,3.315476e+07,ETH
...,...,...,...,...,...,...,...
2021-12-15 16:00:00,8.327,8.327,8.233,8.233,13726.490173,1.141282e+05,FLOW
2021-12-15 17:00:00,8.243,8.270,8.231,8.246,14843.769471,1.223179e+05,FLOW
2021-12-15 18:00:00,8.271,8.362,8.271,8.361,738.473554,6.152375e+03,FLOW


In [82]:
ft = pd.pivot_table(df, values='close', index='ts', columns=['token'],)

In [83]:
ft.head()

token,AVAX,ETH,FLOW,SOL,USDT
ts,,,,,
2021-11-22 00:00:00,128.67,4262.99,14.150,230.785,1.0012
2021-11-22 01:00:00,134.85,4234.37,13.621,226.000,1.0013
2021-11-22 02:00:00,132.54,4217.89,13.753,227.037,1.0012
2021-11-22 03:00:00,133.38,4193.47,13.652,223.117,1.0011
2021-11-22 04:00:00,136.29,4168.35,13.581,220.652,1.0009


In [125]:
lag_return=ft.apply(lambda x:np.log(x)-np.log(x.shift(1)))

In [126]:
lag_return.head()

token,AVAX,ETH,FLOW,SOL,USDT
ts,,,,,
2021-11-22 00:00:00,NaN,NaN,NaN,NaN,NaN
2021-11-22 01:00:00,0.046912,-0.006736,-0.038102,-0.020952,0.0001
2021-11-22 02:00:00,-0.017279,-0.003900,0.009644,0.004578,-0.0001
2021-11-22 03:00:00,0.006318,-0.005806,-0.007371,-0.017417,-0.0001
2021-11-22 04:00:00,0.021583,-0.006008,-0.005214,-0.011109,-0.0002
